In [1]:
import sys
!{sys.executable} -m pip install category_encoders

  Using cached category_encoders-2.8.1-py3-none-any.whl.metadata (7.9 kB)
  Using cached patsy-1.0.1-py2.py3-none-any.whl.metadata (3.3 kB)
  Using cached statsmodels-0.14.5-cp311-cp311-win_amd64.whl.metadata (9.8 kB)
Using cached category_encoders-2.8.1-py3-none-any.whl (85 kB)
Using cached patsy-1.0.1-py2.py3-none-any.whl (232 kB)
   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.6 MB 653.6 kB/s eta 0:00:15
    --------------------------------------- 0.1/9.6 MB 1.0 MB/s eta 0:00:10
    --------------------------------------- 0.1/9.6 MB 847.9 kB/s eta 0:00:12
    --------------------------------------- 0.2/9.6 MB 827.9 kB/s eta 0:00:12
    --------------------------------------- 0.2/9.6 MB 827.9 kB/s eta 0:00:12
    --------------------------------------- 0.2/9.6 MB 758.5 kB/s eta 0:00:13
   - -------------------------------------- 0.3/9.6 MB 863.3 

In [9]:
import sys
!{sys.executable} -m pip install liac-arff

  Using cached liac_arff-2.5.0-py3-none-any.whl


In [2]:
import arff
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from category_encoders import TargetEncoder

In [92]:
with open(r"I:\Mihini\SLIIT\Y2S1\AIMl-Project\Student-depression\depression_dataset.arff", 'r') as f:
    dataset = arff.load(f)

In [93]:
df = pd.DataFrame(dataset['data'], columns=[a[0] for a in dataset['attributes']])

In [79]:
df.head()

,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,2.0,Male,33.0,Visakhapatnam,Student,5.0,0.0,8.97,2.0,0.0,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1.0
1,8.0,Female,24.0,Bangalore,Student,2.0,0.0,5.90,5.0,0.0,5-6 hours,Moderate,BSc,No,3.0,2.0,Yes,0.0
2,26.0,Male,31.0,Srinagar,Student,3.0,0.0,7.03,5.0,0.0,Less than 5 hours,Healthy,BA,No,9.0,1.0,Yes,0.0
3,30.0,Female,28.0,Varanasi,Student,3.0,0.0,5.59,2.0,0.0,7-8 hours,Moderate,BCA,Yes,4.0,5.0,Yes,1.0
4,32.0,Female,25.0,Jaipur,Student,4.0,0.0,8.13,3.0,0.0,5-6 hours,Moderate,M.Tech,Yes,1.0,1.0,No,0.0


In [94]:
le_gender = LabelEncoder()
df['Gender'] = le_gender.fit_transform(df['Gender'])

In [95]:
te_city = TargetEncoder()
df['City'] = te_city.fit_transform(df['City'], df['Depression'])

In [96]:
profession_counts = df['Profession'].value_counts()
print(profession_counts)

Profession
Student                   27870
Architect                     8
Teacher                       6
Digital Marketer              3
Content Writer                2
Chef                          2
Doctor                        2
Pharmacist                    2
Civil Engineer                1
UX/UI Designer                1
Educational Consultant        1
Manager                       1
Lawyer                        1
Entrepreneur                  1
Name: count, dtype: int64


In [97]:
prof_counts = df['Profession'].value_counts()

rare_prof = prof_counts[prof_counts < 10].index
df['Profession'] = LabelEncoder().fit_transform(df['Profession'].apply(lambda x: 'Other' if x in rare_prof else x))

In [98]:
print(df['Sleep Duration'].unique())
print(df['Sleep Duration'].value_counts())

['5-6 hours' 'Less than 5 hours' '7-8 hours' 'More than 8 hours' 'Others']
Sleep Duration
Less than 5 hours    8310
7-8 hours            7346
5-6 hours            6183
More than 8 hours    6044
Others                 18
Name: count, dtype: int64


In [99]:
df['Sleep Duration'] = df['Sleep Duration'].replace('Others', 'More than 8 hours')

sleep_order = ['Less than 5 hours', '5-6 hours', '7-8 hours', 'More than 8 hours']
ordinal_sleep = OrdinalEncoder(categories=[sleep_order])
df['Sleep Duration'] = ordinal_sleep.fit_transform(df[['Sleep Duration']])

In [100]:
print(df['Dietary Habits'].unique())
print(df['Dietary Habits'].value_counts())

['Healthy' 'Moderate' 'Unhealthy' 'Others']
Dietary Habits
Unhealthy    10317
Moderate      9921
Healthy       7651
Others          12
Name: count, dtype: int64


In [101]:
df['Dietary Habits'] = df['Dietary Habits'].replace('Others', 'Moderate')

diet_order = ['Unhealthy', 'Moderate', 'Healthy']
ordinal_diet = OrdinalEncoder(categories=[diet_order])
df['Dietary Habits'] = ordinal_diet.fit_transform(df[['Dietary Habits']])

In [102]:
te_degree = TargetEncoder()
df['Degree'] = te_degree.fit_transform(df['Degree'], df['Depression'])

In [103]:
le_suicide = LabelEncoder()
df['Have you ever had suicidal thoughts ?'] = le_suicide.fit_transform(df['Have you ever had suicidal thoughts ?'])

In [104]:
le_family = LabelEncoder()
df['Family History of Mental Illness'] = le_family.fit_transform(df['Family History of Mental Illness'])

In [105]:
encoded_columns = [
    'Gender', 'City', 'Profession', 'Sleep Duration', 
    'Dietary Habits', 'Degree', 'Have you ever had suicidal thoughts ?', 'Family History of Mental Illness'
]

print(df[encoded_columns + ['Depression']].head())

   Gender      City  Profession  Sleep Duration  Dietary Habits    Degree  \
0       1  0.569659           1             1.0             2.0  0.528395   
1       0  0.608866           1             1.0             1.0  0.588964   
2       1  0.556122           1             0.0             2.0  0.535000   
3       0  0.531387           1             2.0             1.0  0.571528   
4       0  0.524131           1             1.0             1.0  0.509785   

   Have you ever had suicidal thoughts ?  Family History of Mental Illness  \
0                                      1                                 0   
1                                      0                                 1   
2                                      0                                 1   
3                                      1                                 1   
4                                      1                                 0   

   Depression  
0         1.0  
1         0.0  
2         0.0  
3   

In [106]:
print(df.dtypes)

id                                       float64
Gender                                     int32
Age                                      float64
City                                     float64
Profession                                 int32
Academic Pressure                        float64
Work Pressure                            float64
CGPA                                     float64
Study Satisfaction                       float64
Job Satisfaction                         float64
Sleep Duration                           float64
Dietary Habits                           float64
Degree                                   float64
Have you ever had suicidal thoughts ?      int32
Work/Study Hours                         float64
Financial Stress                         float64
Family History of Mental Illness           int32
Depression                               float64
dtype: object


In [108]:
df.head(20)

,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,2.0,1,33.0,0.569659,1,5.0,0.0,8.97,2.0,0.0,1.0,2.0,0.528395,1,3.0,1.0,0,1.0
1,8.0,0,24.0,0.608866,1,2.0,0.0,5.90,5.0,0.0,1.0,1.0,0.588964,0,3.0,2.0,1,0.0
2,26.0,1,31.0,0.556122,1,3.0,0.0,7.03,5.0,0.0,0.0,2.0,0.535000,0,9.0,1.0,1,0.0
3,30.0,0,28.0,0.531387,1,3.0,0.0,5.59,2.0,0.0,2.0,1.0,0.571528,1,4.0,5.0,1,1.0
4,32.0,0,25.0,0.524131,1,4.0,0.0,8.13,3.0,0.0,1.0,1.0,0.509785,1,1.0,1.0,0,0.0
5,33.0,1,29.0,0.558884,1,2.0,0.0,5.70,3.0,0.0,0.0,2.0,0.547893,0,4.0,1.0,0,0.0
6,52.0,1,30.0,0.590869,1,3.0,0.0,9.54,4.0,0.0,2.0,2.0,0.588964,0,1.0,2.0,0,0.0
7,56.0,0,30.0,0.596610,1,2.0,0.0,8.04,4.0,0.0,0.0,0.0,0.707730,0,0.0,1.0,1,0.0
8,59.0,1,28.0,0.557604,1,3.0,0.0,9.79,1.0,0.0,2.0,1.0,0.546867,1,12.0,3.0,0,1.0
9,62.0,1,31.0,0.528336,1,2.0,0.0,8.38,3.0,0.0,0.0,1.0,0.530551,1,2.0,5.0,0,1.0
